In [3]:
import pandas as pd
import numpy as np
import sqlite3
import io
import numpy.ma as ma
def adapt_array(arr):
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

def convert_array(text):
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)

sqlite3.register_adapter(np.ndarray, adapt_array)
sqlite3.register_converter("array", convert_array)


conn = sqlite3.connect("chemdatabase_copy.db", detect_types=sqlite3.PARSE_DECLTYPES)
cursor = conn.cursor()


In [2]:
import yaml

with open("config_endpoints.yml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile)


In [3]:
endpoints_list = cfg["endpoints_list"]

In [4]:
endpoints_list

['o_mus_ipr_LD',
 'o_rat_orl_TDLo',
 'o_mus_ipr_LDLo',
 'o_mus_orl_TDLo',
 'o_rat_ipr_TDLo']

In [5]:
select * from endpoints 

SyntaxError: invalid syntax (<ipython-input-5-3c3b79463d1a>, line 1)

In [6]:
cursor.execute("select * from experimental_dat where ")

OperationalError: incomplete input

In [16]:
from create_database import open_database

In [17]:
open_database(cfg["main"]["db_name"])

(<sqlite3.Connection at 0x7f694a0d92d0>, <sqlite3.Cursor at 0x7f694a1e1960>)

In [23]:
cfg['endpoints_list']

['o_mus_ipr_LD',
 'o_rat_orl_TDLo',
 'o_mus_ipr_LDLo',
 'o_mus_orl_TDLo',
 'o_rat_ipr_TDLo']

In [73]:
!pwd

/home/workstation/Frolov/DBChem


In [3]:

for section in cfg:
    print(section)
print(cfg['mysql'])
print(cfg['other'])

mysql
other
{'host': 'localhost', 'user': 'root', 'passwd': 'my secret password', 'db': 'write-math'}
{'preprocessing_queue': ['preprocessing.scale_and_center', 'preprocessing.dot_reduction', 'preprocessing.connect_lines'], 'use_anonymous': True}


In [4]:
cfg['mysql']

{'host': 'localhost',
 'user': 'root',
 'passwd': 'my secret password',
 'db': 'write-math'}

In [7]:
cfg['preprocessing_queue']

KeyError: 'preprocessing_queue'

In [8]:
cfg

{'mysql': {'host': 'localhost',
  'user': 'root',
  'passwd': 'my secret password',
  'db': 'write-math'},
 'other': {'preprocessing_queue': ['preprocessing.scale_and_center',
   'preprocessing.dot_reduction',
   'preprocessing.connect_lines'],
  'use_anonymous': True}}

In [7]:
temp =cursor.execute("select * from endpoints").fetchall()
temp = [(data[0], data[1].strip()+"_"+data[2].strip()) for data in temp]
NameOfEndpoint2db_index = {data[1]:data[0] for data in temp}


In [32]:
NameOfEndpoint2db_index['o_mus_ipr_LD']

1

In [ ]:
['o_mus_ipr_LD']

In [8]:
indexes = [NameOfEndpoint2db_index[endpoint] for endpoint in cfg['endpoints_list']]

In [9]:
indexes

[1, 2, 3, 4, 5]

In [10]:
"("+", ".join(str(x) for x in indexes )+")"

'(1, 2, 3, 4, 5)'

In [11]:
temp =cursor.execute("select * from experimental_data where id_endpoint in "+("("+", ".join(str(x) for x in indexes )+")")).fetchall()

In [12]:
temp[:5]

[(992, 1, 4.41),
 (10518, 1, 3.99),
 (61767, 1, 3.8),
 (44985, 1, 3.8),
 (34192, 1, 3.49)]

In [13]:
endpoints_dict = {}

In [14]:
for data in temp:
    if data[0] in endpoints_dict.keys():
        endpoints_dict[data[0]].append((data[1],data[2]))
    else:
        endpoints_dict[data[0]] = [(data[1],data[2])]

In [15]:
indexe = [x+1  for x in list(range(len(endpoints_list)))]

In [16]:
for key in endpoints_dict.keys():
    vals = {idx: None for idx in indexe}
    for elem in endpoints_dict[key]:
        vals[elem[0]] = elem[1]
    endpoints_dict[key] = list(vals.values())

In [17]:
db_smiles = cursor.execute("select * from molecules").fetchall()

In [18]:
index2smiles = {smile[0]: smile[3] for smile in db_smiles}

In [19]:
endpoints_keys = list(endpoints_dict.keys())
for key in endpoints_keys:
    endpoints_dict[index2smiles[key]] = endpoints_dict.pop(key)

In [20]:
endpoints_list

['o_mus_ipr_LD',
 'o_rat_orl_TDLo',
 'o_mus_ipr_LDLo',
 'o_mus_orl_TDLo',
 'o_rat_ipr_TDLo']

In [21]:
df = pd.DataFrame.from_dict(endpoints_dict,orient="index", columns=endpoints_list)

In [22]:
df_tox = pd.read_csv("toxicity_85832.csv")

In [27]:
df_tox = df_tox.set_index("SMILES", )

In [28]:
df_tox.head()

,o_mus_ipr_LD,o_rat_orl_TDLo,o_mus_ipr_LDLo,o_mus_orl_TDLo,o_rat_ipr_TDLo,o_mus_ivn_LD,o_rat_ipr_LD,o_mus_orl_LD,o_mus_unr_LD,o_rat_unr_LD,...,o_rbt_orl_LD,o_rbt_ivn_LD,o_rat_ivn_LD,o_mus_orl_LDLo,o_rat_skn_LD,o_mam_unr_LD,o_gpg_orl_LD,o_wmn_orl_TDLo,o_man_orl_TDLo,o_rat_scu_TDLo
SMILES,,,,,,,,,,,,,,,,,,,,,
Br.Br.Br.Br.N=C(N)SCCN(CCSC(=N)N)CCSC(=N)N,4.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Br.Br.Br.CCN(CCSC(=N)N)CCSC(=N)N,3.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Br.Br.Br.CN(CCSC(=N)N)CCSC(=N)N,3.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Br.Br.Br.CNc1cnc(C)c(N)c1CSC(=N)N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.04,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Br.Br.Br.COc1ccc2[nH]cc(CCNC(=O)C(N)CSSCC(N)C(=O)NCCc3c[nH]c4ccc(OC)cc34)c2c1,NaN,NaN,NaN,NaN,NaN,4.27,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df_tox.loc["Br.Br.Br.Br.N=C(N)SCCN(CCSC(=N)N)CCSC(=N)N"]

o_mus_ipr_LD       4.41
o_rat_orl_TDLo      NaN
o_mus_ipr_LDLo      NaN
o_mus_orl_TDLo      NaN
o_rat_ipr_TDLo      NaN
o_mus_ivn_LD        NaN
o_rat_ipr_LD        NaN
o_mus_orl_LD        NaN
o_mus_unr_LD        NaN
o_rat_unr_LD        NaN
o_mus_scu_LDLo      NaN
o_rat_scu_LD        NaN
o_mus_scu_LD        NaN
o_rat_ipr_LDLo      NaN
o_mus_ipr_TDLo      NaN
o_rbt_skn_LD        NaN
o_rat_orl_LD        NaN
o_rat_ivn_TDLo      NaN
o_rat_orl_LDLo      NaN
o_rbt_orl_LD        NaN
o_rbt_ivn_LD        NaN
o_rat_ivn_LD        NaN
o_mus_orl_LDLo      NaN
o_rat_skn_LD        NaN
o_mam_unr_LD        NaN
o_gpg_orl_LD        NaN
o_wmn_orl_TDLo      NaN
o_man_orl_TDLo      NaN
o_rat_scu_TDLo      NaN
Name: Br.Br.Br.Br.N=C(N)SCCN(CCSC(=N)N)CCSC(=N)N, dtype: float64

In [30]:
df.loc["Br.Br.Br.Br.N=C(N)SCCN(CCSC(=N)N)CCSC(=N)N"]

o_mus_ipr_LD      4.41
o_rat_orl_TDLo     NaN
o_mus_ipr_LDLo     NaN
o_mus_orl_TDLo     NaN
o_rat_ipr_TDLo     NaN
Name: Br.Br.Br.Br.N=C(N)SCCN(CCSC(=N)N)CCSC(=N)N, dtype: float64

In [31]:
import numpy.ma as ma

In [32]:
class OurRobustToNanScaler():
    """
    This class is equal to StandardScaler from sklearn but can work with NaN's (ignoring it) but
    sklearn's scaler can't do it.
    """
    def fit(self, data):
        masked = ma.masked_invalid(data)
        self.means = np.mean(masked, axis=0)
        self.stds = np.std(masked, axis=0)
    
    def fit_transform(self, data):
        self.fit(data)
        masked = ma.masked_invalid(data)
        masked -= self.means
        masked /= self.stds
        return ma.getdata(masked)

    def inverse_transform(self, data):
        masked = ma.masked_invalid(data)
        masked *= self.stds
        masked += self.means
        return ma.getdata(masked)

In [33]:
df.values

array([[4.41,  nan,  nan,  nan,  nan],
       [3.99,  nan,  nan,  nan,  nan],
       [3.8 ,  nan,  nan,  nan,  nan],
       ...,
       [2.83,  nan,  nan,  nan,  nan],
       [2.26,  nan,  nan,  nan,  nan],
       [2.7 ,  nan,  nan,  nan,  nan]])

In [34]:
scaler = OurRobustToNanScaler()

In [35]:
scaler.fit(df.values)

In [36]:
new_vals = scaler.fit_transform(df.values)

In [37]:
new_vals

array([[ 1.96798376,         nan,         nan,         nan,         nan],
       [ 1.36189809,         nan,         nan,         nan,         nan],
       [ 1.08771648,         nan,         nan,         nan,         nan],
       ...,
       [-0.31205281,         nan,         nan,         nan,         nan],
       [-1.13459764,         nan,         nan,         nan,         nan],
       [-0.49965075,         nan,         nan,         nan,         nan]])

In [39]:
new_vals

array([[ 1.96798376,         nan,         nan,         nan,         nan],
       [ 1.36189809,         nan,         nan,         nan,         nan],
       [ 1.08771648,         nan,         nan,         nan,         nan],
       ...,
       [-0.31205281,         nan,         nan,         nan,         nan],
       [-1.13459764,         nan,         nan,         nan,         nan],
       [-0.49965075,         nan,         nan,         nan,         nan]])

In [40]:
df[:][:] = new_vals

In [41]:
df.head()

,o_mus_ipr_LD,o_rat_orl_TDLo,o_mus_ipr_LDLo,o_mus_orl_TDLo,o_rat_ipr_TDLo
Br.Br.Br.Br.N=C(N)SCCN(CCSC(=N)N)CCSC(=N)N,1.967984,NaN,NaN,NaN,NaN
Br.Br.Br.CCN(CCSC(=N)N)CCSC(=N)N,1.361898,NaN,NaN,NaN,NaN
Br.Br.Br.CN(CCSC(=N)N)CCSC(=N)N,1.087716,NaN,NaN,NaN,NaN
Br.Br.Br.N=C(N)SCCCNCCNCC12CC3CC(CC(C3)C1)C2,1.087716,NaN,NaN,NaN,NaN
Br.Br.Br.N=C(N)SCCNCCSC(=N)N,0.640368,NaN,NaN,NaN,NaN


In [44]:
smiles_ind = list(set([data[0] for data in temp]))

In [45]:
descriptors = cursor.execute("select * from descriptors_values where id_molecule in "+("("+", ".join(str(x) for x in smiles_ind )+")")).fetchall()

In [46]:
ind2descriptors = {index2smiles[data[0]]:np.nan_to_num(data[4]) for data in descriptors}

In [49]:
smiles2descriptors = {index2smiles[data[0]]:np.nan_to_num(data[4])[0] for data in descriptors}

In [52]:
arr_smiles_descriptors = list(smiles2descriptors.values())

In [57]:
vstacked_smiles_descriptors = np.vstack(arr_smiles_descriptors)

In [58]:
some_test = vstacked_smiles_descriptors[20340]

In [60]:
smile = list(smiles2descriptors.keys())[20340]

In [61]:
all(smiles2descriptors[smile]==some_test)

True

In [ ]:
vstacked_smiles_descriptors

In [213]:
df.head()

,o_mus_ipr_LD,o_rat_orl_TDLo,o_mus_ipr_LDLo,o_mus_orl_TDLo,o_rat_ipr_TDLo
Br.Br.Br.Br.N=C(N)SCCN(CCSC(=N)N)CCSC(=N)N,1.967984,NaN,NaN,NaN,NaN
Br.Br.Br.CCN(CCSC(=N)N)CCSC(=N)N,1.361898,NaN,NaN,NaN,NaN
Br.Br.Br.CN(CCSC(=N)N)CCSC(=N)N,1.087716,NaN,NaN,NaN,NaN
Br.Br.Br.N=C(N)SCCCNCCNCC12CC3CC(CC(C3)C1)C2,1.087716,NaN,NaN,NaN,NaN
Br.Br.Br.N=C(N)SCCNCCSC(=N)N,0.640368,NaN,NaN,NaN,NaN


In [228]:
df_descriptors = pd.DataFrame.from_dict(smiles2descriptors, orient="index")

KeyboardInterrupt: 

In [62]:
import pickle

In [65]:
with open("dickt.pk", 'wb') as f:
    pickle.dump(smiles2descriptors, f)

In [66]:
del smiles2descriptors

In [67]:
with open("dickt.pk", 'rb') as f:
    smiles2descriptors = pickle.load(f)

In [69]:
smiles2descriptors["Br.Br.Br.Br.N=C(N)SCCN(CCSC(=N)N)CCSC(=N)N"]

array([13.384261, 12.092147,  0.      , ..., 78.      ,  0.      ,
        4.5     ], dtype=float32)

In [70]:
df.head()

,o_mus_ipr_LD,o_rat_orl_TDLo,o_mus_ipr_LDLo,o_mus_orl_TDLo,o_rat_ipr_TDLo
Br.Br.Br.Br.N=C(N)SCCN(CCSC(=N)N)CCSC(=N)N,1.967984,NaN,NaN,NaN,NaN
Br.Br.Br.CCN(CCSC(=N)N)CCSC(=N)N,1.361898,NaN,NaN,NaN,NaN
Br.Br.Br.CN(CCSC(=N)N)CCSC(=N)N,1.087716,NaN,NaN,NaN,NaN
Br.Br.Br.N=C(N)SCCCNCCNCC12CC3CC(CC(C3)C1)C2,1.087716,NaN,NaN,NaN,NaN
Br.Br.Br.N=C(N)SCCNCCSC(=N)N,0.640368,NaN,NaN,NaN,NaN


In [78]:
%load_ext autoreload


In [79]:
%autoreload 2

In [4]:
from get_data import get_data

In [5]:
df, smiles2descriptors = get_data("config_endpoints.yml")

In [4]:
from get_data import OurRobustToNanScaler

In [6]:
df.head()

,o_mus_ipr_LD,o_rat_orl_TDLo,o_mus_ipr_LDLo,o_mus_orl_TDLo,o_rat_ipr_TDLo,o_mus_ivn_LD,o_rat_ipr_LD,o_mus_orl_LD,o_mus_unr_LD,o_rat_unr_LD,...,o_rbt_orl_LD,o_rbt_ivn_LD,o_rat_ivn_LD,o_mus_orl_LDLo,o_rat_skn_LD,o_mam_unr_LD,o_gpg_orl_LD,o_wmn_orl_TDLo,o_man_orl_TDLo,o_rat_scu_TDLo
Br.Br.Br.Br.N=C(N)SCCN(CCSC(=N)N)CCSC(=N)N,4.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Br.Br.Br.CCN(CCSC(=N)N)CCSC(=N)N,3.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Br.Br.Br.CN(CCSC(=N)N)CCSC(=N)N,3.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Br.Br.Br.CNc1cnc(C)c(N)c1CSC(=N)N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.04,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Br.Br.Br.COc1ccc2[nH]cc(CCNC(=O)C(N)CSSCC(N)C(=O)NCCc3c[nH]c4ccc(OC)cc34)c2c1,NaN,NaN,NaN,NaN,NaN,4.27,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
smiles2descriptors["Br.Br.Br.Br.N=C(N)SCCN(CCSC(=N)N)CCSC(=N)N"]

array([13.384261, 12.092147,  0.      , ..., 78.      ,  0.      ,
        4.5     ], dtype=float32)

In [7]:
smiles = list(df.index)

In [8]:
y = df.values

In [9]:
y.shape

(85832, 29)

In [10]:
x = np.vstack(list(smiles2descriptors.values()))

In [243]:
temp_df = pd.read_csv("db_endpoints.csv")

In [ ]:
temp_df

In [251]:
smiles_temp = list(temp_df[temp_df.columns[0]])

In [257]:
del temp_df[temp_df.columns[0]]

In [259]:
temp_df.values

array([[4.41,  nan,  nan, ...,  nan,  nan,  nan],
       [3.99,  nan,  nan, ...,  nan,  nan,  nan],
       [3.8 ,  nan,  nan, ...,  nan,  nan,  nan],
       ...,
       [2.26,  nan,  nan, ...,  nan,  nan,  nan],
       [2.7 ,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan]])

In [11]:
x.shape

(85832, 1826)

In [151]:
class OurRobustToNanScaler():
    """
    This class is equal to StandardScaler from sklearn but can work with NaN's (ignoring it) but
    sklearn's scaler can't do it.
    """

    def fit(self, data):
        masked = ma.masked_invalid(data)
        self.means = np.mean(masked, axis=0)
        self.stds = np.std(masked, axis=0)

    def fit_transform(self, data):
        self.fit(data)
        masked = ma.masked_invalid(data)
        masked -= self.means
        masked /= self.stds
        return ma.getdata(masked)

    def inverse_transform(self, data):
        masked = ma.masked_invalid(data)
        masked *= self.stds
        masked += self.means
        return ma.getdata(masked)


In [152]:
input_scaler = OurRobustToNanScaler()
output_scaler = OurRobustToNanScaler()


In [240]:
import pickle

In [241]:
with open("input_scaler.pk", 'wb') as f:
    pickle.dump(input_scaler, f)

In [242]:
with open("output_scaler.pk", 'wb') as f:
    pickle.dump(input_scaler, f)

In [203]:
x = np.load("morded112.npy")

In [179]:
smiles[0]

'Br.Br.Br.Br.N=C(N)SCCN(CCSC(=N)N)CCSC(=N)N'

In [181]:
from descriptors import morded

In [182]:
morded(smiles[0])

array([[13.384261, 12.092147,  0.      , ..., 78.      ,  0.      ,
         4.5     ]], dtype=float32)

In [188]:
smiles2descriptors[smiles[0]]

array([13.384261, 12.092147,  0.      , ..., 78.      ,  0.      ,
        4.5     ], dtype=float32)

In [186]:
(x[0,:]==morded(smiles[0])).all()

True

In [187]:
(x[-1,:]==morded(smiles[-1])).all()

True

In [204]:
x = input_scaler.fit_transform(x)

In [210]:
y = df.values

In [211]:
y = output_scaler.fit_transform(y)

In [ ]:
df.to_csv("df")

In [ ]:
from 

In [212]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
class LinearLayer(nn.Module):
    def __init__(self, size_in, size_out, p=0.5):
        super(LinearLayer, self).__init__()
        self.size_in = size_in
        self.size_out = size_out
        self.p = p
        self.fc = nn.Linear(self.size_in, self.size_out)
        self.dropout = nn.Dropout(self.p)
        self.relu = nn.ReLU()
        self.batchnorm = nn.BatchNorm1d(size_out)
        nn.init.xavier_uniform_(self.fc.weight)
        self.fc.bias.data.fill_(0.01)
    def forward(self, x):
        return self.dropout(self.relu(self.batchnorm(self.fc(x))))
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear1 = LinearLayer(1825, 512, 0.5)
        self.linear2 = LinearLayer(512, 256, 0.5)
        self.linear3 = LinearLayer(256, 128, 0.5)
        self.linear4 = LinearLayer(128, 64, 0.5)
        self.linear5 = LinearLayer(64, 32, 0.5)
        self.linear6 = LinearLayer(32, 29, 0.1)
        self.linear7 = LinearLayer(29, 29, 0.0)
        self.layers = [self.linear1,self.linear2, self.linear3, self.linear4, self.linear5,self.linear6,self.linear7]
    def forward(self, x):
        output = x
        for layer in self.layers:
            output = layer(output)
        return output
device = torch.device('cuda:0')
model = MLP().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.00005)

In [213]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [195]:
X_train[0].shape

(1825,)

In [214]:
from torch.utils.data import Dataset, DataLoader

In [215]:
class ToxicDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.mask = ~ma.masked_invalid(self.y).mask
        self.y = np.nan_to_num(self.y)
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return (torch.from_numpy(self.x[idx]),torch.from_numpy(self.y[idx]),torch.from_numpy(self.mask[idx]))

train_dataset = ToxicDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=512, shuffle=True)


test_dataset = ToxicDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=512, shuffle=False)


In [217]:
log_file = open("log.txt", 'w')

In [218]:
def train(epoch):
    model.train()
    losses = []
    for batch_idx, (x, y, mask) in enumerate(train_loader):
        #x = x.reshape(-1, max_len*512)
        x, y, mask = x.to(device,), y.to(device), mask.to(device)
        x = x.float()
        y = y.float()
        mask = mask.float()
        optimizer.zero_grad()
        output = model(x)
        output.squeeze_(dim=1)
        
        loss_func = nn.MSELoss()
        loss= loss_func(mask*output, mask*y)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()

        #if batch_idx % 1000 == 0:
        #    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #        epoch, batch_idx * len(x), len(train_loader.dataset),
        #        100. * batch_idx / len(train_loader), sum(losses)/len(losses)),flush=True)
    return sum(losses)/len(losses)
def test(epoch):
    model.eval()
    losses = []
    for batch_idx, (x, y, mask) in enumerate(test_loader):
        x, y, mask = x.to(device), y.to(device), mask.to(device)
        x = x.float()
        y = y.float()
        mask = mask.float()
        
        # print("x_shape: ", x.shape)
        # pеrint("y_shape: ", y.shape)

        output = model(x)
        output.squeeze_(dim=1)
        # print(output.shape)
        # print(y.shape)
        # print(output)
        # print(y)

        loss_func = nn.MSELoss()
        loss = loss_func(mask*output, mask*y)
        losses.append(loss.item())

        #if batch_idx % 200 == 0:
        #    print('Test Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #        epoch, batch_idx * len(x), len(test_loader.dataset),
        #               100. * batch_idx / len(test_loader), sum(losses) / len(losses)),flush=True)
    return sum(losses)/len(losses)
for epoch in range(1, 5000 + 1):
    loss = train(epoch)
    log_file.write("Epoch{} ".format(epoch))
    log_file.write(str(loss))
    log_file.write("\n")
    if epoch%10==0:
        torch.save(model.state_dict(), "ckpt/mlp_epoch{}.pt".format(epoch))
    loss = test(epoch)
    log_file.write("Epoch{} ".format(epoch))
    log_file.write(str(loss))
    log_file.write("\n")
    log_file.flush()

In [61]:
for batch_idx, (x, y, mask) in enumerate(train_loader):
    #x = x.reshape(-1, max_len*512)
    x, y, mask = x.to(device,), y.to(device), mask.to(device)
    x = 
    mask = mask.double()
    print(x.dtype)
    print(y.dtype)
    print(mask.dtype)
    print((mask*x).dtype)
    break

torch.float32
torch.float64
torch.float64


RuntimeError: The size of tensor a (29) must match the size of tensor b (1826) at non-singleton dimension 1

In [ ]:

def test_eval(epoch):
    model.eval()
    losses = []
    for batch_idx, (x, y, mask) in enumerate(test_loader):
        x, y, mask = x.to(device), y.to(device), mask.to(device)
        x = x.float()
        y = y.float()
        mask = mask.float()
        
        # print("x_shape: ", x.shape)
        # pеrint("y_shape: ", y.shape)

        output = model(x)
        output.squeeze_(dim=1)
        # print(output.shape)
        # print(y.shape)
        # print(output)
        # print(y)

        loss_func = nn.MSELoss()
        loss = loss_func(mask*output, mask*y)
        losses.append(loss.item())

        #if batch_idx % 200 == 0:
        #    print('Test Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #        epoch, batch_idx * len(x), len(test_loader.dataset),
        #               100. * batch_idx / len(test_loader), sum(losses) / len(losses)),flush=True)
    return sum(losses)/len(losses)

In [226]:
model.eval()
x_s = []
y_s = []
masks = []
outputs =[]
for batch_idx, (x, y, mask) in enumerate(test_loader):
    x, y, mask = x.to(device), y.to(device), mask.to(device)
    x = x.float()
    y = y.float()
    mask = mask.float()
    x_s.append(x.detach().cpu().numpy())
    y_s.append(y.detach().cpu().numpy())
    masks.append(mask.detach().cpu().numpy())
    output = model(x)
    outputs.append(output.detach().cpu().numpy())

In [227]:
x_s = np.vstack(x_s)
y_s = np.vstack(y_s)
masks = np.vstack(masks)
outputs = np.vstack(outputs)

In [228]:
masks = np.array(masks, dtype= bool)

In [235]:
y_s[~masks] = np.nan

outputs[~masks]= np.nan

y_s = output_scaler.inverse_transform(y_s)

output_scaler = output_scaler.inverse_transform(outputs)

In [236]:
def mse(y_true, y_pred):
    return np.mean((y_true- y_pred)**2)

In [237]:
mse_for_diff_endpoints = []
for i in range(y_s.shape[1]):
    mse_for_diff_endpoints.append(mse((masks*y_s)[:, i][np.array(masks[:,i], dtype = bool)],(masks*outputs)[:, i][np.array(masks[:,i], dtype = bool)]) )

In [266]:
from sklearn.metrics import r2_score
r2score_for_diff_endpoints = []
for i in range(y_s.shape[1]):
    r2score_for_diff_endpoints.append(r2_score((masks*y_s)[:, i][np.array(masks[:,i], dtype = bool)],(masks*outputs)[:, i][np.array(masks[:,i], dtype = bool)]) )

In [132]:
mse_for_diff_endpoints = []
for i in range(y_s.shape[1]):
    mse_for_diff_endpoints.append(mse((masks*y_s)[:, i],(masks*outputs)[:, i]) )

In [267]:
r2score_for_diff_endpoints

[-15.513023563271297,
 -8.230352505945678,
 -14.914942654907145,
 -14.442126030348946,
 -13.323702946088114,
 -21.610752091413385,
 -9.346887701028985,
 -16.072077409680627,
 -8.497139378350447,
 -7.42666014616781,
 -11.815667771500086,
 -5.423776724669832,
 -10.120581121183442,
 -17.880442631354576,
 -12.659827467641493,
 -6.277373315336609,
 -6.338308309636158,
 -10.61027340724174,
 -6.866735046183721,
 -8.71164993234454,
 -8.982397131851993,
 -9.550969927775812,
 -11.504351814935207,
 -7.355407185670087,
 -15.475211992231834,
 -3.9984337230669382,
 -9.75831398550138,
 -7.314356951420379,
 -11.951533530902887]

In [271]:
with open("mse_r2.txt", "w") as f:
    for type, mse, r2_score_val in zip (list(df.columns), mse_for_diff_endpoints, r2score_for_diff_endpoints):
        f.write(type+" "+ str(mse)+" "+str(r2_score_val)+"\n")

In [238]:
mse_for_diff_endpoints

[8.379019,
 16.341059,
 9.7987,
 18.195963,
 20.781704,
 13.091166,
 9.100436,
 5.973465,
 8.024556,
 7.618749,
 10.012424,
 8.63042,
 9.259889,
 9.937033,
 20.191845,
 3.9741356,
 5.773755,
 30.02434,
 6.8971095,
 6.062716,
 18.081434,
 13.501424,
 6.8952975,
 5.169603,
 6.8628,
 7.353192,
 18.700832,
 17.808025,
 27.85087]

In [94]:
(masks*y_s)[:, 0].shape

(17167,)

In [93]:
mse_for_diff_endpoints

[2.4599009,
 4.1211667,
 3.254926,
 4.346537,
 4.654204,
 3.38631,
 3.1354077,
 2.2443123,
 3.0488715,
 2.991999,
 3.2503185,
 3.1215994,
 3.0746663,
 3.3013237,
 4.6194735,
 2.09643,
 2.458603,
 5.550138,
 2.8488564,
 2.604922,
 4.387722,
 3.8550618,
 2.8000438,
 2.4127805,
 2.7471344,
 2.9232857,
 4.420727,
 4.3541155,
 5.3316245]

In [79]:
masks

array([[0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [239]:
model

MLP(
  (linear1): LinearLayer(
    (fc): Linear(in_features=1825, out_features=512, bias=True)
    (dropout): Dropout(p=0.5)
    (relu): ReLU()
    (batchnorm): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (linear2): LinearLayer(
    (fc): Linear(in_features=512, out_features=256, bias=True)
    (dropout): Dropout(p=0.5)
    (relu): ReLU()
    (batchnorm): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (linear3): LinearLayer(
    (fc): Linear(in_features=256, out_features=128, bias=True)
    (dropout): Dropout(p=0.5)
    (relu): ReLU()
    (batchnorm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (linear4): LinearLayer(
    (fc): Linear(in_features=128, out_features=64, bias=True)
    (dropout): Dropout(p=0.5)
    (relu): ReLU()
    (batchnorm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (linear5): LinearLayer(
    (fc): 